In [ ]:
! pip3 install bayesian_optimization
! pip3 install cython
! sudo apt-get update
! sudo apt-get -y install build-essential
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
import cython
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%load_ext Cython

In [38]:
random_state = 54949393

In [22]:
train_values_short = pd.read_csv('train_values_short1.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')
proba_grado_correcto = pd.read_csv('proba_grado_correcto.csv', index_col='building_id')

In [23]:
%%cython
def hassanat1(double[:] x, double[:] y, double[:] factores):
    cdef int n = x.shape[0]
    cdef double res = 0
    cdef double d, minimo, maximo

    for i in range(n):
        minimo = min(x[i],y[i])
        maximo = max(x[i],y[i])
        if (minimo >=0):
            d =  1 - ( (1+minimo)/(1+maximo) )
        else:
            d =  1 - ( (1+minimo+abs(minimo))/(1+maximo+abs(minimo)) )
        res += d * factores[i]
    
    return res

In [24]:
def target(resultado):
    return resultado['target']

def mostrar(resultados, keys, min=0):
    
    print( "i", end = '\t\t') 
    print("target", end = '\t\t') 
    for key in keys:
        print(  key, end = '\t') 
    print()
    
    a_borrar = []
    for resultado in resultados:
        if(resultado['target']<min):
            a_borrar.append(resultado)
    resultados_importantes = [a for a in resultados if a not in a_borrar]
    
    resultados_importantes.sort(reverse=True, key=target)
    
    for i, res in enumerate(resultados_importantes):
        cells = []

        cells.append(i)
        cells.append(res["target"])

        for key in keys:
            cells.append(res["params"][key])
        
        for j in cells:
            print("| " + str(j), end = '\t') 
            
        print()

In [34]:
x_train, x_test, y_train, y_test = train_test_split(train_values_short,
                                train_labels, test_size=0.05, random_state=random_state)

In [28]:
factores = np.array([1.82264756, 0.14128378, 0.27999595, 0.4755266 , 2.30883141,
       1.02996122, 1.07966807, 1.10292129, 0.83811159, 0.80736305,
       0.74243681, 4.93845144, 4.93845144, 7.76639614, 0.03964061550504756, 0.09684727174529291, 0.025805063269526796])\
        .astype('double')

In [35]:
def funcion_a_optimizar(max_a_borrar):
    a_borrar = proba_grado_correcto.loc[proba_grado_correcto['proba_grado_correcto'] < max_a_borrar, :].index
    x_train_filtered = x_train.copy(deep=True)
    y_train_filtered = y_train.copy(deep=True)
    x_train_filtered.drop(index=a_borrar, inplace=True, errors='ignore'  )
    y_train_filtered.drop(index=a_borrar, inplace=True, errors='ignore'  )
    knn = KNeighborsClassifier(weights="distance",n_neighbors=22,
                  metric=hassanat1, metric_params={'factores':factores})
    knn.fit(x_train_filtered, y_train_filtered.values.ravel())
    prediction = knn.predict(x_test)
    return f1_score(y_test, prediction, average='micro')

In [39]:
limites = {"max_a_borrar":(0,0.5001)}

optimizer = BayesianOptimization(
    f=funcion_a_optimizar,
    pbounds=limites,
    random_state=random_state,
    bounds_transformer=SequentialDomainReductionTransformer(),
    verbose=5,
)
str_random_state = "%s" %random_state
#load_logs(optimizer, "./anteriores_" + str_random_state  + ".json")
logger = JSONLogger(path="./logs_nuevos_" + str_random_state  + ".json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [41]:
optimizer.maximize(
    init_points=2,
    n_iter=5,
)

67240


In [40]:
optimizer.probe(
    params={'max_a_borrar':0},
    lazy=True,
)

In [ ]:
optimizer.probe(
    params={'max_a_borrar':0.5001},
    lazy=True,
)